In [39]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [40]:
file = pd.read_csv("F:\DEEP_LEARNING\FINAL\CONCATENADO_COMPLETO.csv", encoding='latin-1')
#file_cleaned = file.dropna().drop_duplicates().reset_index(drop=True).rename(columns={'description':'title'})
file_cleaned = file.rename(columns={'TEXTO':'title'})


In [41]:
file_cleaned

,title
0,FALTA DE TRABAJO PANDEMIA NC Perderá puestos d...
1,ECONOMIA PANDEMIA NS Mantendrá el mismo número...
2,COVID FALTA DE LIQUIDES DESABASTO DE ENTREGA M...
3,LA ECONOMIA DEL PAIS LOS CREDITOS QUE NO NOS D...
4,Economia Pandemia Inseguridad Perderá puestos ...
...,...
1697,LA COBRANZA NC NC Mantendrá el mismo número de...
1698,LA ECONOMÍA. NS NS Perderá puestos de trabajo ...
1699,ROTACION DE PERSONAL CAPACITACIÓN NC Creará nu...
1700,ECONOMICO NS NS Mantendrá el mismo número de p...


In [42]:
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)
    # Estas son mias
    
    #text = sub(r" ", "_", text)
    
    text = sub(r" que ", " ", text)
    text = sub(r" la ", " ", text)
    text = sub(r"la ", " ", text)
    text = sub(r" del ", " ", text)
    text = sub(r" de ", " ", text)
    text = sub(r" es ", " ", text)
    text = sub(r" mia ", " ", text)
    text = sub(r" los ", " ", text)
    text = sub(r"los ", " ", text)
    text = sub(r" las ", " ", text)
    text = sub(r"las ", " ", text)
    text = sub(r" el ", " ", text)
    text = sub(r"el ", " ", text)
    #text = sub(r"no", " ", text)
    text = sub(r" por ", " ", text)
    text = sub(r" se ", " ", text)
    text = sub(r" para ", " ", text)
    text = sub(r" es ", " ", text)
    text = sub(r" esta ", " ", text)
    text = sub(r" ese ", " ", text)
    text = sub(r" aquel ", " ", text)
    #text = sub(r" aquella ", " ", text)
    text = sub(r" que  ", " ", text)
    text = sub(r" y ", " ", text)
    text = sub(r" con ", " ", text)
    text = sub(r" a ", " ", text)
    text = sub(r" en ", " ", text)
    text = sub(r" nos ", " ", text)
    text = sub(r" con ", " ", text)
    #text = sub(r" como ", " ", text)
    text = sub(r" al ", " ", text)
    text = sub(r" por ", " ", text)
    #text = sub(r" estan ", " ", text)
    text = sub(r" lo ", " ", text)
    text = sub(r" se ", " ", text)
    #text = sub(r" ya ", " ", text)
    
    text = text.split()

    return text

In [43]:
file_cleaned.title = file_cleaned.title.apply(lambda x: text_to_word_list(x, unidecode))

In [44]:
file_cleaned.title

0       [falta, trabajo, pandemia, nc, perdera, puesto...
1       [economia, pandemia, ns, mantendra, mismo, num...
2       [covid, falta, liquides, desabasto, entrega, m...
3       [economia, pais, creditos, no, dan, pandemia, ...
4       [economia, pandemia, inseguridad, perdera, pue...
                              ...                        
1697    [cobranza, nc, nc, mantendra, mismo, numero, p...
1698    [economia, ns, ns, perdera, puestos, trabajo, ...
1699    [rotacion, personal, capacitacion, nc, creara,...
1700    [economico, ns, ns, mantendra, mismo, numero, ...
1701    [economia, actividad, menor, np, mantendra, mi...
Name: title, Length: 1702, dtype: object

In [45]:
file_cleaned.title[17]

['alza',
 'costo',
 'actual',
 'viviendo',
 'insumos',
 'innecesarios',
 'siembra',
 'cultivo',
 'pademia',
 'bajaron',
 'muchisimo',
 'ventas',
 'pedidos',
 'clientes',
 'cuanto',
 'seguro',
 'social',
 'todo',
 'eso',
 'no',
 'hay',
 'ni',
 'clinica',
 'mantendra',
 'mismo',
 'numero',
 'puestos',
 'trabajo',
 'esp',
 'no',
 'fue',
 'menor',
 'menor',
 'proveedores',
 'opera',
 'empresa',
 'han',
 'mantenido',
 'han',
 'incorporado',
 'nuevos',
 'satisfacen',
 'totalmente',
 'costos',
 'superiores',
 'si',
 'si',
 'no',
 'si',
 'ns',
 'disminuiran',
 'ns',
 'permaneceran',
 'igual',
 'disminuiran',
 '98',
 'ns',
 'ns',
 'le',
 'fue',
 'igual',
 'o',
 'mas',
 'o',
 'menos',
 'igual',
 'no',
 'tengo',
 'conocimiento',
 'mal',
 'trimestre',
 'bueno',
 'negativo',
 'no',
 'mejor',
 'va',
 'mejorar',
 'poco',
 'no',
 'va',
 'salir',
 'muy',
 'bueno',
 'no',
 'beneficio',
 'ns',
 'muy',
 'bueno',
 'yo',
 'creo',
 'tener',
 'un',
 'poco',
 'empatia',
 'no',
 'pueden',
 'hacer',
 'pagos',
 '

In [46]:
file_model = file_cleaned.copy()
#file_model = file_model[file_model.title.str.len()>1]
file_model = file_model[file_model.title.str.len()>0]

In [47]:
file_model

,title
0,"[falta, trabajo, pandemia, nc, perdera, puesto..."
1,"[economia, pandemia, ns, mantendra, mismo, num..."
2,"[covid, falta, liquides, desabasto, entrega, m..."
3,"[economia, pais, creditos, no, dan, pandemia, ..."
4,"[economia, pandemia, inseguridad, perdera, pue..."
...,...
1697,"[cobranza, nc, nc, mantendra, mismo, numero, p..."
1698,"[economia, ns, ns, perdera, puestos, trabajo, ..."
1699,"[rotacion, personal, capacitacion, nc, creara,..."
1700,"[economico, ns, ns, mantendra, mismo, numero, ..."


In [48]:
sent = [row for row in file_model.title]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[21]

INFO - 11:42:47: collecting all words and their counts
INFO - 11:42:47: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 11:42:47: collected 24375 token types (unigram + bigrams) from a corpus of 130040 words and 1702 sentences
INFO - 11:42:47: merged Phrases<24375 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 11:42:47: Phrases lifecycle event {'msg': 'built Phrases<24375 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000> in 0.22s', 'datetime': '2021-05-01T11:42:47.412542', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'created'}
INFO - 11:42:47: exporting phrases from Phrases<24375 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 11:42:47: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<987 phrases, min_count=1, threshold=10.0> from Phrases<24375 vocab, min_count=1, threshold=10.0, m

['falta',
 'trabajo',
 'falta_recursos',
 'economia',
 'no',
 'estable',
 'mantendra_mismo',
 'numero_puestos',
 'trabajo',
 'esp',
 'no',
 'fue',
 'menor',
 'menor',
 'empresa_dejo',
 'contar_proveedores',
 'venia_operando',
 'no',
 'han_sido',
 'sustituidos_nuevos',
 'falta',
 'trabajo',
 'nada',
 'comprometida',
 'si',
 'si',
 'no',
 'no',
 'no',
 'disminuiran',
 '50',
 'permaneceran',
 'igual',
 'aumentaran',
 '50',
 '2',
 'nc',
 'le',
 'fue',
 'peor',
 'si',
 'tengo_conocimiento',
 'muy',
 'mal_trimestre',
 'muy',
 'malo',
 'negativo',
 'no',
 'peor',
 'va',
 'empeorar',
 'poco',
 'no',
 'va',
 'salir',
 'bueno',
 'no',
 'beneficio',
 'ns',
 'bueno',
 'esperar_pagos',
 'trabajores',
 'regule',
 'pais']

min count = 3 - remove most unusual words from training embeddings, like words 'ssssuuuuuuuppppppeeeeeerrrr', which actually stands for 'super', and doesn't need additional training
window = 4 - Word2Vec model will learn to predict given word from up to 4 words to the left, and up to 4 words to the right
size = 300 - size of hidden layer used to predict surroundings of embedded word, which also stands for dimensions of trained embeddings
sample = 1e-5 - probability baseline for subsampling most frequent words from surrounding of embedded word
negative = 20 - number of negative (ones that shouldn't have been predicted while modeling selected pair of words) words that will have their corresponding weights updated while training on specific training example, along with positive word

In [50]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     #size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 11:43:09: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-05-01T11:43:09.514003', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'created'}
INFO - 11:43:09: collecting all words and their counts
INFO - 11:43:09: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:43:09: collected 5207 word types from a corpus of 112099 raw words and 1702 sentences
INFO - 11:43:09: Creating a fresh vocabulary
INFO - 11:43:09: Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 1555 unique words (29.863645093143845%% of original 5207, drops 3652)', 'datetime': '2021-05-01T11:43:09.664602', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'prepare_vocab'}
INFO - 11:43:09: Wo

Time to build vocab: 0.0 mins


In [51]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=200, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 11:43:17: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 1555 vocabulary and 100 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4', 'datetime': '2021-05-01T11:43:17.086644', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'train'}
INFO - 11:43:17: worker thread finished; awaiting finish of 6 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 5 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 4 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 3 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 2 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 1 more threads
INFO - 11:43:17: worker thread finished; awaiting finish of 0 more threads
INFO - 11:43:17: EPOCH - 1 : training on 112099 raw words (8569 effective words) took 0

Time to train the model: 0.54 mins


In [52]:
w2v_model.save("F:\DEEP_LEARNING\FINAL\word2vec.model")

INFO - 11:43:49: Word2Vec lifecycle event {'fname_or_handle': 'F:\\DEEP_LEARNING\\FINAL\\word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-05-01T11:43:49.451532', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'saving'}
INFO - 11:43:49: not storing attribute cum_table
INFO - 11:43:50: saved F:\DEEP_LEARNING\FINAL\word2vec.model


In [53]:
file_export = file_model.copy()
file_export['old_title'] = file_export.title
file_export.old_title = file_export.old_title.str.join(' ')
file_export.title = file_export.title.apply(lambda x: ' '.join(bigram[x]))
#file_export.rate = file_export.rate.astype('int8')

In [54]:
file_export[['title']].to_csv('F:\DEEP_LEARNING\FINAL\cleaned_dataset.csv', index=False)

AQUI SE HACE EL CLUSTER

In [55]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [56]:
word_vectors = Word2Vec.load("F:\DEEP_LEARNING\FINAL\word2vec.model").wv

#word_vectors = Word2Vec.load("F:\DEEP_LEARNING\FINAL\word2vec.model")

INFO - 11:43:52: loading Word2Vec object from F:\DEEP_LEARNING\FINAL\word2vec.model
INFO - 11:43:52: loading wv recursively from F:\DEEP_LEARNING\FINAL\word2vec.model.wv.* with mmap=None
INFO - 11:43:52: setting ignored attribute cum_table to None
INFO - 11:43:52: Word2Vec lifecycle event {'fname': 'F:\\DEEP_LEARNING\\FINAL\\word2vec.model', 'datetime': '2021-05-01T11:43:52.200295', 'gensim': '4.0.1', 'python': '3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.17763-SP0', 'event': 'loaded'}


In [57]:
word_vectors.vectors.size

155500

In [58]:
word_vectors.vectors

array([[ 0.17244335,  0.02840863,  0.01379685, ..., -0.0988087 ,
        -0.02140561, -0.01366452],
       [-0.02342317,  0.05027406,  0.14896037, ...,  0.0088178 ,
        -0.15484756,  0.08719335],
       [ 0.20224155,  0.10922083, -0.04831236, ...,  0.03199477,
         0.1369488 , -0.00182759],
       ...,
       [-0.08492719,  0.15957361, -0.00265899, ...,  0.10043229,
        -0.07614505,  0.01658797],
       [ 0.07071864,  0.08271409,  0.06969794, ...,  0.06065789,
         0.03326107, -0.05172023],
       [-0.04381762,  0.17467296, -0.03561446, ..., -0.11653829,
        -0.00909085,  0.0022132 ]], dtype=float32)

In [59]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [60]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=30, restrict_vocab=None)

[('amables', 0.9121060371398926),
 ('actualizado', 0.9104927778244019),
 ('llegar', 0.9086015224456787),
 ('dia', 0.9073891639709473),
 ('medio', 0.8960623145103455),
 ('hacen', 0.8934348821640015),
 ('me_parece', 0.8933690190315247),
 ('opcion', 0.8931028246879578),
 ('misma', 0.8901859521865845),
 ('alguien', 0.8897815346717834),
 ('infonavit', 0.8888742327690125),
 ('ofrecen', 0.8886940479278564),
 ('empleado', 0.8864032030105591),
 ('realmente', 0.883570671081543),
 ('infonat', 0.8834506273269653),
 ('creo_ya', 0.8812314867973328),
 ('un_credito', 0.8792811632156372),
 ('veces', 0.878207802772522),
 ('tienen', 0.8779926300048828),
 ('les', 0.8775001168251038),
 ('trabajador', 0.8774752616882324),
 ('usted', 0.8758722543716431),
 ('esa_parte', 0.875474750995636),
 ('tanto_tiempo', 0.8752546310424805),
 ('corriente', 0.8748846650123596),
 ('dicen', 0.8738439083099365),
 ('lugar', 0.8721133470535278),
 ('esto', 0.87143474817276),
 ('vamos', 0.8710911870002747),
 ('cuando', 0.870724201

In [61]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=30, restrict_vocab=None)

[('contigencia', 0.9008308053016663),
 ('otorgar_prorrogas', 0.8918442726135254),
 ('alza_precios', 0.8851232528686523),
 ('tiempo_forma', 0.8801649212837219),
 ('s', 0.8770817518234253),
 ('gastos_fijos', 0.8754327297210693),
 ('baja_turismo', 0.8701605200767517),
 ('cambios', 0.8680880069732666),
 ('venta', 0.8647578358650208),
 ('cierres', 0.8595678210258484),
 ('demanda', 0.8534142374992371),
 ('limpieza_higiene', 0.852030873298645),
 ('baja_venta', 0.8514607548713684),
 ('brindar_prorrogas', 0.8502992391586304),
 ('aumento_insumos', 0.8491955995559692),
 ('zona', 0.84918212890625),
 ('baja_demanda', 0.8472987413406372),
 ('personal_capacitado', 0.8459280729293823),
 ('altos_impuestos', 0.8456833362579346),
 ('proyectos', 0.845590353012085),
 ('necesidades_empresas', 0.8442854881286621),
 ('covid_19', 0.8441399335861206),
 ('economica', 0.8400495052337646),
 ('ausentismo', 0.8376584053039551),
 ('control_personal', 0.8359262943267822),
 ('locales', 0.8349251747131348),
 ('facil_cub

In [62]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [63]:
#words = pd.DataFrame(word_vectors.vocab.keys())

words = pd.DataFrame(word_vectors.key_to_index.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [64]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [65]:
words[['words', 'sentiment_coeff']].to_csv('F:\DEEP_LEARNING\FINAL\sentiment_dictionary.csv', index=False)

Aqui metemos la prediccion

In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

In [28]:
final_file = pd.read_csv('F:\DEEP_LEARNING\FINAL\cleaned_dataset.csv')

In [29]:
sentiment_map = pd.read_csv('F:\DEEP_LEARNING\FINAL\sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [30]:
file_weighting = final_file.copy()

In [31]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(file_weighting.title)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(file_weighting.title)

C:\Users\IN335255\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [32]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.title.split()))

In [33]:
%%time
replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

Wall time: 561 ms


Aqui calculamos el score de sentimiento

In [34]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [35]:
replaced_closeness_scores = file_weighting.title.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

Hacemos las predicciones

In [36]:
#replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.title, file_weighting.rate]).T
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.title]).T

#replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']


replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
#replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
#replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]

In [37]:
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment_rate
0,"[-1.4154076014677686, -1.5064415745079405, -1....","[2.4191233313131617, 5.467507852811911, 6.3581...",falta trabajo pandemia nc perdera_puestos trab...,83.723450
1,"[-1.5437228413973492, -1.5113873178224142, -1....","[3.6118329433603873, 2.119378386412106, 3.6549...",economia pandemia ns mantendra_mismo numero_pu...,-349.535762
2,"[-1.6501905021150909, -1.6433432101613237, -1....","[4.58999907895263, 5.349104227304372, 6.137561...",covid falta_liquides desabasto entrega mantend...,-304.900449
3,"[-1.5537856622067618, 1.1905504188732323, -1.3...","[6.360705138982852, 3.3902906734131513, 11.045...",economia_pais creditos no dan pandemia mantend...,-255.219380
4,"[-1.5437228413973492, -1.5113873178224142, -1....","[3.6118329433603873, 2.119378386412106, 4.6789...",economia pandemia inseguridad perdera_puestos ...,-243.370766
...,...,...,...,...
1697,"[-1.4332978603197593, -1.1095600630738889, -1....","[5.495707701496248, 3.1257012183305193, 3.1257...",cobranza nc nc mantendra_mismo numero_puestos ...,-224.125677
1698,"[-1.5437228413973492, -1.0834835836450796, -1....","[3.6118329433603873, 7.309865132245435, 7.3098...",economia ns ns perdera_puestos trabajo no fue ...,-293.852897
1699,"[-1.4705159151194518, -1.1457351474430968, -1....","[4.109413340376357, 5.144309814658359, 6.25140...",rotacion_personal capacitacion nc creara_nuevo...,-270.678368
1700,"[-1.4420677437349592, -1.0834835836450796, -1....","[4.8566277422065784, 6.0915542768711965, 6.091...",economico ns ns mantendra_mismo numero_puestos...,-219.853710


In [38]:
replacement_df.to_csv(r'F:\DEEP_LEARNING\FINAL\resultados_1.csv', index=False)

In [536]:
predicted_classes = replacement_df.prediction
y_test = replacement_df.sentiment

conf_matrix = pd.DataFrame(confusion_matrix(replacement_df.sentiment, replacement_df.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

AttributeError: 'DataFrame' object has no attribute 'prediction'